In [ ]:
#Rev. 0.1 original various iterations, to track a celestial source
#Date: 11/2/2021
#Rev. 0.2 - remove ability to add az/el offsets from within this script
# in order to track a number of sources and save the logger files per source automatically
# set duration of each track through global constant
#Rev. d: Add schedule from file function, clear SCU program track prior to each source, date: 08 Sep 2021
#Rev. e Add proper motion correction and katpoint V1.0a1 compatible, add refraction EL limit date: 28 Sep 2021
#Rev. f Update ACU log and RFC file naming 07 Oct 2021

In [ ]:
import sculib
from cam_sensors import sensor_data_pvsn
import datetime
import numpy as np
import katpoint
from astropy.time import Time
from astropy import units as u  
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.table import Table
import pandas as pd 
import time
from pathlib import Path
    
mpi_lat = '-30:43:04.719'
mpi_lon = '21:24:46.933'
mpi_alt = 1086.0

In [ ]:
mpi = sculib.scu()
mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '192.168.101.136'
scu = sim                 #!!!!!!!!!!<<<<<<<<<<<<<<<<< Set to mpi for Antenna

In [ ]:
def ska_unwrap_tt(t, az, el):
    #Basic unwrap azimuth to be negative if az angle larger than 270
###    uaz = np.array(list(map(lambda x: x-360 if x > 270 else x, az ))) 
    uaz = az
    #removed limits here, its a hard life for a new dish
    ###########added margin for limits
    
    '''
    :#Test code
    az = range(0,361,10)
    unwrap_az = map(lambda x: x-360 if x > 270 else x, az )
    '''
    return(t, uaz, el)

def optimize_tt(t, az, el, current_az):
    # Try to make best use of -270 to +270 deg azimuth range wrap and take shortest route to the next source  
    oaz = az # by default use as is
###################################################    
    if current_az >= 0: # current encoder >0
        if az[0] < 270: #new is within cw range
            if (az[0] - current_az) <= 180: # shortest route is cw
                oaz = az
            else: # cw not the shortest route
                oaz = az - 360
        else: # outside cw range\n",
            oaz = az - 360
    else: # current encoder <0
        if az[0] > 90: #new is within CCW range
            if (current_az - az[0]) <= -180: # shortest route is ccw
                oaz = az - 360
            else: # ccw not the shortest route
                oaz = az 
        else: # outside ccw range
            oaz = az
    
    # Check for a zero azimuth crossing in the table and correct individual entries no to jump from 0 to 365
    i=1
    while i <= len(oaz)-1:
        x = oaz[i]
        dx = oaz[i]-oaz[i-1]
        if dx >= 10: # wrapping through zero (10 chosen arbitrary to catch the jump)
            oaz[i] = oaz[i] - 360
        i = i+1    
###################################################            
    return(t, oaz, el)

def add_refract_tt(t, az, el, r_temp, r_press, r_rh, o_lat, o_lon, o_alt, Tstart):
    '''
    The refraction model is based on that implemented in ERFA, which is fast but becomes inaccurate for altitudes 
    below about 5 degrees. Near and below altitudes of 0, it can even give meaningless answers, and in this case 
    transforming to AltAz and back to another frame can give highly discrepant results. For much better numerical 
    stability, leaving the pressure at 0 (the default), disabling the refraction correction (yielding 
    “topocentric” horizontal coordinates).
    
    altitude for elevation and height for altitude, really?
    '''
    #Add elevation limit to avoid refraction correction mathematical errors
    
    elelim = 89
    k=0
    for c in el:
        if c > elelim:
            el[k]=elelim  
        k=k+1
    
    
    #owl is observation wavelength, optical 300 to 700nm
    owl = 0.00000055*u.m  #owl*u.m
    
    #astropy need to know the units
    temp = r_temp*u.Celsius
    presr = r_press/1000*u.bar
    RH=r_rh/100    #humidity is relative to 1
    
    #user in ERFA CRD
    loc = EarthLocation.from_geodetic(lat=o_lat, lon=o_lon, height=o_alt*u.m, ellipsoid='WGS84')
    T0 = Time(t[0], format='mjd') # Irrelevant, but necessary for astropy transformations
    
    #mother function, has scaling and T0
    erfa_crd = lambda ele, sc, presr, temp, RH : AltAz(alt=ele*u.deg, az=0*u.deg, location=loc, obstime=T0).transform_to( AltAz(obstime=T0, location=loc, obswl=owl, pressure=sc*presr, temperature=sc*temp, relative_humidity=sc*RH))
    
    #calls mother with two sc of 1 and 0 the convert to degrees?    
    erfa = lambda ele, presr, temp, RH: (erfa_crd(ele,1,presr,temp,RH).alt - erfa_crd(ele,0,presr,temp,RH).alt).deg # deg to add to unrefracted altitude

    refco = erfa(el, presr, temp, RH) # deg
    rel = el + refco

    #define output file data and parameters and save the file
    data = {'Time':  t,
                'Az': az,
                'El': el,
                'refco': refco,
                'rEl': rel,
                'temp': [temp]*len(t),
                'presr': [presr]*len(t),
                'RH': [RH]*len(t),
                }
    
    df = pd.DataFrame (data, columns = ['Time', 'Az', 'El', 'refco', 'rEl', 'temp', 'presr', 'RH'])
    
    file_name =  '6p13p2_RFC_'+ str(Tstart) + '_HIP'+str(target)+'.csv'
    file_path = Path.cwd()  / 'output' / file_name
    
    df.to_csv (file_path, index = False, header=True)
    
    return(t, az, rel) 

def gen_target_tt(observer, target, UTC_start, duration, dt=0.1, verbose=False, fname=None):
    """
        Convert J2000 RA,DEC to topocentric Az,El angles at time intervals as specified.
        Note: Topocentric angle is the apparent position computed for an observer at a specific location 
        on the surface of the earth, including corrections for geocentric parallax, diurnal abberation and 
        Earth rotation angle but neglecting atmospheric refraction.
        
        Example:
            R2D = 180/np.pi
            SDQM = katpoint.Antenna("SDQM", latitude=-30.7172/R2D, longitude=21.4129/R2D, altitude=1053.0) 
            # TODO: lat & lon
            gen_target_tt(SDQM, ra = "05:34:30", dec = "22:00:57", UTC_start = datetime.datetime(2019,10,14,8,0,0), 
                            duration = 300, dt = 0.1, verbose = True)
            
        @param observer: [katpoint.Antenna]
        @param target: either a string (then tgt_name) or tuple (then ra,dec) as below
        @param tgt_name: target name for SIMBAR query [string]
        @param ra : [string or float] Right ascension, either in 'H:M:S' or decimal degree string format, 
                     or as a float in radians.
        @param dec : [string or float] Declination, either in 'D:M:S' or decimal degree string format,
                     or as a float in radians.
        @param UTC_start: now as datetime time of first coordinate in UTC
        @param duration: [float] duration of the track table, now in seconds not hours ppak 1/7/2021.
        @param dt: [float] interval between coordinates in the track table, in seconds.
        @return: [list of tuples: (time, Az, El)] with time no in MJD ; 
                 topocentric Az in degrees East of North; topocentric El in degrees above the horizon.
    """
    #Conversion Radian to Degrees
    R2D = 180/np.pi  
    
    #Get target details Hipparcos VOT table.
    tgt = hip[hip['HIP'] == target]
    
    #Convert target into SkyCoord object
    tgtSC = SkyCoord(ra=tgt['_RAJ2000'], dec=tgt['_DEJ2000'],pm_ra_cosdec=tgt['pmRA'], pm_dec=tgt['pmDE'],obstime=Time('J2000'))
    
    #Apply Proper Motion (Space motion without radial motion)
    tnow = Time.now()    #Time used to apply proper motion relative to obstime (J2000), not really needed to be this accurate
    tgtSM = tgtSC.apply_space_motion(new_obstime=tnow)
    tgtPM = SkyCoord(tgtSM.realize_frame(tgtSM.data.without_differentials())) #required to avoid UnitConversionError due radial velocity
    
    #Convert to a Katpoint target
    tgtKP = katpoint.Target(tgtPM)
    
    #Generate Time vector
    T0 = UTC_start.timestamp()
    T = T0 + np.arange(0, duration, dt)
    
    #calculate az & el positions using Katpoint
    azel = tgtKP.azel(T, observer)
    taz = azel.az.deg       #Az corrected for proper motion
    tel = azel.alt.deg      #El corrected for proper motion
    
    return(Time(T, format='unix').mjd, taz, tel)

def get_enviro(T0, duration):
    '''
    returns mean of temp, pres, humidity from T0-duration until T0
    T0: time now as datetime
    duration: how far back in past as seconds you want to get values of temp, pres, humidity
    '''
    #use receptor m000, it does not matter which one you use all receptors use the same temperature stations
    receptor = 'm000'
    
    #list of sensors used for reference, below we query them directly
    sensors=['_enviro_air_pressure',
             '_enviro_air_relative_humidity',
             '_enviro_air_temperature']

    #get time now, needed to request temp, press and relative humidity
    timestop=T0.timestamp()
    
    #time start for sensor retrieval as far back as we are going forward
    timestart=(T0 - datetime.timedelta(seconds=duration)).timestamp()

    #sensor_date_pvsn returns timestamp data as well, but not used
    timestampv,timestamps,air_temp=sensor_data_pvsn(receptor+'_enviro_air_temperature', timestart, timestop)
    mean_air_temp=np.mean(air_temp)

    timestampv,timestamps,air_press=sensor_data_pvsn(receptor+'_enviro_air_pressure', timestart, timestop)
    mean_air_press=np.mean(air_press)

    timestampv,timestamps,air_rh=sensor_data_pvsn(receptor+'_enviro_air_relative_humidity', timestart, timestop)
    mean_air_rh=np.mean(air_rh)

    return (mean_air_temp, mean_air_press, mean_air_rh)

    
def mpi_track_offset(scu, az_offset, el_offset):
    #ok so we have a running track, can we add some offset to it???
    scu.debug = False
    
    print('CURRENT OFFSET')
    print('Az: {} *** El: {} \n'.format(scu.status_Value("acu.tracking.act_stat_offset_value_az"), 
                                     scu.status_Value("acu.tracking.act_stat_offset_value_el")))
    scu.status_Value("acu.tracking.act_stat_offset_value_az")
    scu.status_Value("acu.tracking.act_stat_offset_value_el")
    
    scu.load_static_offset(az_offset, el_offset)
    scu.wait_duration(1)
    print('\n NEW OFFSET')
    print('Az: {} *** El: {}'.format(scu.status_Value("acu.tracking.act_stat_offset_value_az"), 
                                     scu.status_Value("acu.tracking.act_stat_offset_value_el")))

In [ ]:
def mpi_gen_tt(target, start, track_len, Tstart, dt=0.1, current_az=0):
    '''
    target: a recognised name from astropy
    start:  a start time in datetime, default now in UTC
    track_len: how long to track source for in seconds
    observer hardcoded to be ska mpi from lat long defined in global constants.
    returns: track table, unwrapped and with refraction correction.
    '''
    # generate track tables
    #observer_skampi = katpoint.Antenna("SKA_MPI", latitude=mpi_lat, longitude=mpi_lon, altitude=mpi_alt)
    observer_skampi = katpoint.Antenna("SKA_MPI"+","+mpi_lat+","+mpi_lon+","+str(mpi_alt))
    
    T0 = start ############+ datetime.timedelta(seconds=2.0) # add two seconds for the offset point before tracking
  
    print('Target: HIP {} *** Start: {} *** Duration: {:.0f} seconds'.format(target, start, track_len))

    #raw track table as from katpoint
    raw_tt = gen_target_tt(observer_skampi, target, UTC_start=T0, duration=track_len, dt=dt)

######################
    #unwrapped track table as for mpi dish
    unwrap_tt = ska_unwrap_tt(*raw_tt)
    opt_tt = optimize_tt(*unwrap_tt, current_az)
########################

    # Refraction correction as for current sensor values?
    temp, pres, rh = get_enviro(T0, track_len)
    tt = add_refract_tt(*opt_tt, temp, pres, rh, mpi_lat, mpi_lon, mpi_alt, Tstart)
    
    return(tt)

In [ ]:
def mpi_track_source(scu, target, start, track_len, dt):  

    
    # Common setup
    if scu == mpi:
        config_name = 'hn_pointing'
        
    if scu == sim:
        config_name = 'ww_sim'
        
    wait5 = 5
    sampling = 100
    doLog = 1
    
    Tstart = str(int(time.time()))
    
    scu.debug = False 
    print('')
    if doLog == 0:
        print('ACU logger disabled !!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    else:
        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait5)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait5)

    current_az=scu.status_Value('acu.azimuth.p_act')
    print('Current az: ', round(current_az,1))
    tt=mpi_gen_tt(target, start, track_len, Tstart, dt, current_az)
    print('New az: ', round(tt[1][0],1),'  el: ',round(tt[2][0],1))
    scu.acu_ska_track(scu.format_body(*tt))
    scu.wait_duration(wait5)
    scu.wait_duration(track_len)
    
    # This is to allow logging to continue until the track is completed
    print('Waiting for track to finish...')
    while (int(scu.status_Value("acu.tracking.act_pt_end_index_a")) 
           - int(scu.status_Value("acu.tracking.act_pt_act_index_a"))) > 0: 
            time.sleep(5)
            pass
    print('Current track finished')

    if doLog == 1:
        scu.stop_logger()
        scu.wait_duration(wait5)
        print(scu.logger_state())
        print('Extract the data on the SCU @ 10Hz sampling')
        scu.save_session14('6p13p2_ACU_'+ str(Tstart) + '_HIP'+str(target), interval_ms = sampling)

In [ ]:
#Optical pointing tests 
print('Test 6.13.2: Pointing Model Calibration & Blind Pointing')
print('--------------------------------------------------------')
print('')

######################################################
how_long = 180 #in units of seconds
print('Tracking time per source [seconds] : ',how_long)
start_delta = 10 # if this is long the dish first runs in TBD direction
timestep = 1 #1s intervals
file = 'target_sched.csv'   #File containing scheduled targets
catfile = 'hipparcos.vot'   #File containing filtered Hipparcos catalogue (from Vizier) 

print('Track Table time increments [seconds] : ',timestep)
print('')
print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
print('!!!!! Make sure the test number for saving the file names are correct !!!!!!!!!!!!!!!')
print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

sched = pd.read_csv(file) #Read in schedule
hip = Table.read(catfile) #Read in Hipparcos filtered catalogue

targets = sched['Source']

  
##############################################################
##    mpi.load_program_track('LOAD_RESET', 1)  ### hierdie werk nog nie lekker nie


for target in targets: ### mpi/sim #######################
    
    buff_t = datetime.datetime.now(datetime.timezone.utc)
    
    scu.load_program_track(load_type = 'LOAD_RESET', entries = 0)
    scu.acu_ska_track_stoploadingtable()
    
    mpi_track_source(scu, 
                     target,
                     start=datetime.datetime.now(datetime.timezone.utc)+datetime.timedelta(seconds=start_delta), 
                     track_len=how_long, 
                     dt=timestep)
    print("Elapsed time per target:",datetime.datetime.now(datetime.timezone.utc)-buff_t)

print('')
print('Done')